This is an example showing how to calculate the critical densities of H2O.

In [1]:
%matplotlib inline

import numpy as np
import os
import sys

for std, __std__ in [
    (sys.stdout, sys.__stdout__),
    (sys.stderr, sys.__stderr__),
]:
    # By default Jupyter notebook displays the terminal output.
    # This is for turning off this feature.
    if getattr(std, "_original_stdstream_copy", None) is not None:
        # redirect captured pipe back to original FD
        os.dup2(std._original_stdstream_copy, __std__.fileno())
        std._original_stdstream_copy = None

def lower_keys(p):
    return {k.lower(): p[k] for k in p}

## Import the wrapper

In [2]:
import wrapper_my_radex
wrapper = wrapper_my_radex.myradex_wrapper
about_info = wrapper.about.tobytes()
column_info = wrapper.column_names.tobytes()

In [3]:
column_names = [_.decode() for _ in wrapper.column_names.item().split()]
print([_ for _ in enumerate(column_names)])

[(0, 'iup'), (1, 'ilow'), (2, 'Eup'), (3, 'freq'), (4, 'lam'), (5, 'Tex'), (6, 'tau'), (7, 'Tr'), (8, 'fup'), (9, 'flow'), (10, 'flux_K'), (11, 'flux_int'), (12, 'flux_Jy'), (13, 'beta'), (14, 'Jnu'), (15, 'gup'), (16, 'glow'), (17, 'Aul'), (18, 'Bul'), (19, 'Blu'), (20, 'n_crit'), (21, 'n_crit_old')]


## Configure the wrapper



In [4]:
transition_dir = os.path.expanduser('~/_c/protoplanetary_disk/transitions/')

In [13]:
n_levels, n_item, n_transitions, n_partners = \
    wrapper.config_basic(dir_transition_rates=transition_dir,
                         filename_molecule='oh2o@rovib.dat',
                         tbg=2.73, verbose=True,
                         max_code_run_time=10, # in seconds
                         max_evol_time=1e7, # in seconds
                         atol=1e-30, rtol=1e-6, solve_method='Newton')

In [14]:
print('Number of energy levels', n_levels)
print('Number of transitions', n_transitions)
print('Number of collisional partners', n_partners)

Number of energy levels 411
Number of transitions 7597
Number of collisional partners 2


## Calculate the critical densities

The meaning of the columns of the array `data_transitions` are included in `column_names`.

In [15]:
params = {'Tkin': 300.0,
          'dv_CGS': 1e5,
          'dens_X_CGS': 1e5,
          'Ncol_X_CGS': 1e17,
          'H2_density_CGS': 1e9,
          'HI_density_CGS': 0.0,
          'oH2_density_CGS': 0.0,
          'pH2_densty_CGS': 0.0,
          'HII_density_CGS': 0.0,
          'Electron_density_CGS': 0.0,
          'n_levels': n_levels,
          'n_item': n_item,
          'n_transitions': n_transitions,
          'donotsolve': False,
          'collisioPartnerCrit': 1,
         }

# The keywords to the wrapper function have to be in lower case,
# so I have to lower the keys of params.
# Of course you can use lower case letters from the beginning.

params = lower_keys(params)

energies,f_occupations,data_transitions,cooling_rate = wrapper.run_one_params(**params)
print(wrapper.flag_good)

0


In [8]:
for i in range(n_transitions):
    lam_micron = 2.99792458e8*1e6/data_transitions[3,i]
    if 14.5 <= lam_micron <= 15.0:
        print('{:4d}'.format(int(data_transitions[0,i])),
              '{:4d}'.format(int(data_transitions[1,i])),
              '{:8.2f}'.format(data_transitions[2,i]),
              '{:6.2f}'.format(lam_micron),
              '{:10.3e}'.format(data_transitions[5,i]),
              '{:10.3e}'.format(data_transitions[19,i]),
              '{:10.3e}'.format(data_transitions[20,i]))

  40   19  1805.90  14.95  3.000e+02  1.914e+07  9.919e+11
  46   24  2030.99  14.98  3.000e+02  3.469e+05  8.149e+08
  52   30  2288.65  14.91  3.000e+02  5.571e+02  1.480e+06
  65   37  2605.03  14.54  3.000e+02  2.707e+08  9.370e+11
  75   43  2876.11  14.88  3.000e+02  5.251e+06  9.491e+09
  91   50  3232.76  14.51  3.000e+02  2.032e+09  1.652e+10
 106   63  3543.52  14.84  3.000e+02  8.185e-01  1.141e+08
 108   65  3589.98  14.60  3.000e+02  2.036e+03  3.308e+11
 120   78  3867.54  14.74  3.000e+02  1.809e+02  7.951e+10
 126   83  3950.91  14.87  3.000e+02  5.846e+07  1.269e+11
 127   83  3965.86  14.64  3.000e+02  3.506e+04  9.322e+07
 132   90  4178.82  14.73  3.000e+02  5.858e+00  1.148e+09
 136   91  4198.70  14.89  3.000e+02  1.135e+10  2.177e+13
 138   91  4212.65  14.68  3.000e+02  4.847e+07  1.162e+11
 146  104  4438.39  14.92  3.000e+02  1.181e+10  3.117e+13
 173  121  4867.76  14.67  3.001e+02  7.786e+02  2.603e+11
 192  135  5168.69  14.78  3.000e+02  2.996e+09  4.424e+

In [12]:
for i in range(n_transitions):
    lam_micron = 2.99792458e8*1e6/data_transitions[3,i]
    if 14.5 <= lam_micron <= 15.0:
        print('{:4d}'.format(int(data_transitions[0,i])),
              '{:4d}'.format(int(data_transitions[1,i])),
              '{:8.2f}'.format(data_transitions[2,i]),
              '{:6.2f}'.format(lam_micron),
              '{:10.3e}'.format(data_transitions[5,i]),
              '{:10.3e}'.format(data_transitions[19,i]),
              '{:10.3e}'.format(data_transitions[20,i]))

  40   19  1805.90  14.95  1.791e+02  1.912e+07  9.906e+11
  46   24  2030.99  14.98  1.452e+02  3.469e+05  8.149e+08
  52   30  2288.65  14.91  1.128e+02  5.571e+02  1.480e+06
  65   37  2605.03  14.54  1.003e+02  2.738e+08  9.475e+11
  75   43  2876.11  14.88  9.824e+01  5.252e+06  9.491e+09
  91   50  3232.76  14.51  1.352e+02  2.078e+09  1.689e+10
 106   63  3543.52  14.84  1.697e+02  8.185e-01  1.141e+08
 108   65  3589.98  14.60  1.920e+02  2.036e+03  3.308e+11
 120   78  3867.54  14.74  3.516e+02  1.809e+02  7.951e+10
 126   83  3950.91  14.87  1.653e+02  5.846e+07  1.270e+11
 127   83  3965.86  14.64  1.366e+02  3.506e+04  9.322e+07
 132   90  4178.82  14.73  4.209e+02  5.858e+00  1.148e+09
 136   91  4198.70  14.89  1.109e+02  1.140e+10  2.187e+13
 138   91  4212.65  14.68  9.571e+01  4.847e+07  1.162e+11
 146  104  4438.39  14.92  1.083e+02  1.184e+10  3.124e+13
 173  121  4867.76  14.67  4.073e+02  7.786e+02  2.603e+11
 192  135  5168.69  14.78  1.696e+02  2.996e+09  4.424e+